Task of this notebook:
1. interpolate data (use tdal data as example... function)... 
    -- Inverse Distance Weighting (gonna try this first and compare with the result produced from R)
    -- spline
    -- krigging

In [ ]:
# libraries
import json
import pandas as pd
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xlrd
from scipy import interpolate
from scipy.interpolate import RectBivariateSpline

In [62]:
# read in PTAL data.
# PTAL = pd.read_csv("/Users/June/urbanDNA/Ele-June/June's Data/COA2011 AvPTAI2015.csv")
# PTAL.head()

In [71]:
# well.. the grid version is more poised for interpolation. #however, it only yields half of London
# hence need to check data source. 
PTAL_grid =pd.read_csv("/Users/June/urbanDNA/Ele-June/June's Data/2015  PTALs Grid Values 280515.csv")
PTAL_grid.head()
# use the Accessiility Index 2015 (AI2015) bc it is numerical variable. 
# PTAL.grid.dtypes
# PTAL.grid.describe()


,ID,X,Y,AI2015,PTAL2015
0,1,530870,155930,0.0,0
1,2,530945,155898,0.0,0
2,3,531045,155898,0.0,0
3,4,531145,155898,0.0,0
4,5,531245,155898,0.0,0


In [72]:
## to put it in a grid.
# PTAL.grid.pivot(index='Y', columns='X', values='AI2015')

In [73]:
# To produce the grid to interplot on
x = np.arange(min(PTAL_grid['X']), max(PTAL_grid['X']), 1)
y = np.arange(min(PTAL_grid['Y']), max(PTAL_grid['Y']), 1)
xx, yy = np.meshgrid(x,y, sparse=True)


In [74]:
# These code are from http://geoexamples.blogspot.co.uk/2012/05/creating-grid-from-scattered-data-using.html
from math import pow
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

# xv, yv, values are all corresponding to the same same dot at the same time. three arrays of the same length.
def pointValue(x,y,power,smoothing,xv,yv,values):
    nominator=0
    denominator=0
    for i in range(0,len(values)):
        dist = sqrt((x-xv[i])*(x-xv[i])+(y-yv[i])*(y-yv[i])+smoothing*smoothing);
        #If the point is really close to one of the data points, return the data point value to avoid singularities
        if(dist<0.0000000001):
            return values[i]
        nominator=nominator+(values[i]/pow(dist,power))
        denominator=denominator+(1/pow(dist,power))
    #Return NODATA if the denominator is zero
    if denominator > 0:
        value = nominator/denominator
    else:
        value = -9999
    return value

# returns value for one grid

def invDist(xv,yv,values,xsize=100,ysize=100,power=2,smoothing=0):
    valuesGrid = np.zeros((ysize,xsize))
    for x in range(0,xsize):
        for y in range(0,ysize):
            valuesGrid[y][x] = pointValue(x,y,power,smoothing,xv,yv,values)
    return valuesGrid

# returns a matrix filled with value calculated from the method above

In [ ]:
power = 1
smoothing=20

#Creating some data, with each coodinate and the values stored in separated lists
xv = PTAL_grid['X']
yv = PTAL_grid['Y']
values = PTAL_grid['AI2015']



#Creating the interpolation function and populating the output matrix value
ZI = invDist(xv,yv,values,len(x),len(y),power,smoothing)


# Plotting the result
n = plt.Normalize(0.0, 100.0)
plt.subplot(1, 1, 1)
plt.pcolor(XI, YI, ZI)
plt.scatter(xv, yv, 100, values)
plt.title('Inv dist interpolation - power: ' + str(power) + ' smoothing: ' + str(smoothing))
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.colorbar()

plt.show()